In [ ]:
!pip install selenium


import sys
!{sys.executable} -m pip install selenium


In [ ]:
!pip install webdriver_manager


In [ ]:
!pip install beautifulsoup4


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pyautogui
from bs4 import BeautifulSoup
import sqlite3
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configuração do Selenium
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

# Configuração do SQLite
conn = sqlite3.connect('questoes2 copy.db')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS questoes (
    id INTEGER PRIMARY KEY,
    materia TEXT,
    submateria TEXT,
    texto_da_questao TEXT,
    opcao_a TEXT,
    opcao_b TEXT,
    opcao_c TEXT,
    opcao_d TEXT,
    opcao_e TEXT
)
''')
conn.commit()


navegador.get("https://projetoagathaedu.com.br/banco-de-questoes.php")

# Loop principal
for i in range(1, 8):

    # Construir o XPath dinamicamente e encontrar o elemento
    xpath = f'/html/body/div[6]/a[{i}]'
    elemento = navegador.find_element('xpath', xpath)
    materia = elemento.text  # Extraia o nome da matéria do texto do link
    navegador.execute_script("arguments[0].scrollIntoView();", elemento)
    elemento.click()
    time.sleep(2)

    # Clique para fechar propaganda se necessário
    pyautogui.click(x=2354, y=391)
    time.sleep(4)

    # Clique para fechar propaganda se necessário
    pyautogui.click(x=2354, y=391)
    time.sleep(3)

    elemento = navegador.find_element('xpath', '/html/body/main/article/div[2]/div/div[1]/div[1]/label')
    navegador.execute_script("arguments[0].scrollIntoView();", elemento)
    elemento.click()
    time.sleep(2)

    # Encontrar a div "painel"

    paineis = navegador.find_elements(By.CLASS_NAME, 'painel')
    for painel in paineis:
        navegador.execute_script("arguments[0].scrollIntoView();", painel)
        
        # for j in range(1,8):
        #     xpath = f'//*[@id="tab_wrapper"]/div[1]/div[2]/div/div/div[{j}]'
        #     elemento = navegador.find_element(By.XPATH, xpath)
        #     elemento_submateria = elemento.find_element(By.CLASS_NAME, 'lista-tema')
        #     submateria = elemento_submateria.text.strip()
        #     navegador.execute_script("arguments[0].scrollIntoView();", elemento)
        #     elemento.click()
        #     time.sleep(2)

        opcoes = painel.find_elements(By.CLASS_NAME, 'opcao')
        
        for opcao in opcoes:
            elemento_submateria = opcao.find_element(By.CLASS_NAME, 'lista-tema')
            submateria = elemento_submateria.text.strip()
            opcao.click()
            

            time.sleep(1)
            
            questoes = navegador.find_elements(By.CLASS_NAME, 'questoes-enem-vestibular')
            
            for questao in questoes:
                html_da_questao = questao.get_attribute('outerHTML')
                soup = BeautifulSoup(html_da_questao, 'html.parser')
                
                if soup.find('img') is None:
                    paragrafos = soup.find_all('p')
                    texto_questao = ' '.join(paragrafo.get_text(strip=True) for paragrafo in paragrafos if 'class' not in paragrafo.attrs or 'questoes-enem-vestibular-foco' not in paragrafo['class'])

                    opcoes = soup.find('ol', type='A')
                    if opcoes:
                        opcoes_texto = []
                        for li in opcoes.find_all('li'):
                            texto = li.get_text(strip=True)
                            # Verifica se o texto da opção contém apenas caracteres simples
                            # (sem marcações HTML complexas como fórmulas matemáticas)
                            if texto.isprintable() and not any(char in texto for char in "<>"):
                                opcoes_texto.append(texto)
                            else:
                                opcoes_texto.append('')

                    # Ajustar a lista de opções para ter sempre 5 elementos
                    opcoes_texto = opcoes_texto[:5] + [''] * (5 - len(opcoes_texto))


                    # Verificar se a questão já existe no banco de dados
                    cursor.execute("SELECT * FROM questoes WHERE materia = ? AND submateria = ? AND texto_da_questao = ?", (materia, submateria, texto_questao))
                    resultado = cursor.fetchone()
                    if resultado is None:
                        cursor.execute("INSERT INTO questoes (materia, submateria, texto_da_questao, opcao_a, opcao_b, opcao_c, opcao_d, opcao_e) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", (materia, submateria, texto_questao, *opcoes_texto))
                        conn.commit()
            
            print('------------------------------------------------------')
            
            navegador.back()
            time.sleep(2)
            painel = navegador.find_element(By.CLASS_NAME, 'painel')



navegador.quit()
conn.close()




------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
----------

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=119.0.6045.124)
Stacktrace:
	GetHandleVerifier [0x005372A3+45731]
	(No symbol) [0x004C2D51]
	(No symbol) [0x003B86D0]
	(No symbol) [0x003ECAC4]
	(No symbol) [0x003E4FC0]
	(No symbol) [0x00407FDC]
	(No symbol) [0x003E4A4E]
	(No symbol) [0x00408254]
	(No symbol) [0x0041B7A2]
	(No symbol) [0x00407DD6]
	(No symbol) [0x003E31F6]
	(No symbol) [0x003E439D]
	GetHandleVerifier [0x00840716+3229462]
	GetHandleVerifier [0x008884C8+3523784]
	GetHandleVerifier [0x0088214C+3498316]
	GetHandleVerifier [0x005C1680+611968]
	(No symbol) [0x004CCCCC]
	(No symbol) [0x004C8DF8]
	(No symbol) [0x004C8F1D]
	(No symbol) [0x004BB2C7]
	BaseThreadInitThunk [0x77857BA9+25]
	RtlInitializeExceptionChain [0x77A2BD3B+107]
	RtlClearBits [0x77A2BCBF+191]


In [ ]:
import pyautogui
import time
time.sleep(3)
pyautogui.position()
